In [1]:
from aip import AipNlp
import pandas as pd
import time
import synonyms

smart_open library not found; falling back to local-filesystem-only
smart_open library not found; falling back to local-filesystem-only


Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt ...
I0816 11:39:42.923965  6176 __init__.py:111] Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\GWT\AppData\Local\Temp\jieba.ua5170a0f831c846f08094f57e4bca895.cache
I0816 11:39:42.926968  6176 __init__.py:131] Loading model from cache C:\Users\GWT\AppData\Local\Temp\jieba.ua5170a0f831c846f08094f57e4bca895.cache


>> Synonyms load wordseg dict [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.617 seconds.
I0816 11:39:44.542110  6176 __init__.py:163] Loading model cost 1.617 seconds.
Prefix dict has been built succesfully.
I0816 11:39:44.543112  6176 __init__.py:164] Prefix dict has been built succesfully.


>> Synonyms on loading stopwords [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\words.vector] ...


In [2]:
""" 你的 APPID AK SK """
APP_ID = '11653919'
API_KEY = 'UvzTwOdwV2viviQ71VoEOigz'
SECRET_KEY = 'RbdwB9RvAAmv3jc4qRyPizeQGY7gWgSt '
client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [3]:
options = {}
options["type"] = 12

In [4]:
f = open('jd行车记录仪评论2018-08-09.csv')
df = pd.read_csv(f)

FileNotFoundError: [Errno 2] No such file or directory: 'jd行车记录仪评论2018-08-09.csv'

In [ ]:
content = df['content']

In [12]:
def geteachtextResult(commentList,errorlen = 0):
    banlist = []
    errorlist = []
    result = []
    print('共有%d条评论'%len(commentList))
    for text in commentList:
        nlp = client.commentTag(text,options)
        print('#',end='')
        if 'error_msg' in nlp:
            if nlp['error_code'] == 282130:
#                 print('不认识')
                errorlist.append(text)
            elif nlp['error_code'] == 18:
#                 print('ban')
                banlist.append(text)
            else:
                 print('未知错误')
                 print(nlp)
        elif 'items' in nlp:
            result.extend(nlp['items'])
        else:
            print('没报错但是也没结果')
            print(nlp)
    if len(banlist)!= 0:
#         a = copy.deepcopy(result)
        result.extend(geteachtextResult(banlist,errorlen+len(errorlist)))
        return result
#         return result.extend(geteachtextResult(banlist,errorlen+len(errorlist),result))
    else:
        print("\n分析完成！%d条没有结果"%(errorlen))
#         a = copy.deepcopy(result)
        return result

In [21]:
testComment = content[50:80]
time1 = time.time()
com = geteachtextResult(testComment)
time2 = time.time()
print(str(time2-time1))

共有30条评论
##############################共有14条评论
##############共有8条评论
########共有5条评论
#####共有4条评论
####
分析完成！10条没有结果
5.373857259750366


In [23]:
result = pd.DataFrame(com)

In [24]:
count = result['prop'].value_counts()
count

感觉      8
商品      2
手机      2
功能      2
确实      2
外观      2
做工      1
超市      1
效果不错    1
物流速度    1
速度快     1
效果      1
物流      1
东西      1
总体      1
质量      1
屏幕      1
种类      1
东西质量    1
速度      1
品牌不错    1
Name: prop, dtype: int64

In [25]:
#同义词
array=list(count.index)
synos=['0']

In [26]:
for i in range(len(array)):
    for j in range(len(synos)):
        if synonyms.compare(array[i], synos[j], seg=True)>0.8:
            break
        else:
            if j == (len(synos)-1):
                synos.append(array[i])
synos.remove('0')

C:\ProgramData\Anaconda3\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))


In [27]:
#好中坏
count=pd.DataFrame(count)

sy_count = pd.DataFrame()
sy_count['prop']=list(count.index)
sy_count['sum']=list(count['prop'])
sy_count['good']=0
sy_count['normal']=0
sy_count['bad']=0

In [28]:
for i in range(len(result)):
    for j in range(len(count)):
        if result['prop'][i]==sy_count['prop'][j]:
            if result['sentiment'][i]==2:
                sy_count['good'][j]=sy_count['good'][j]+1
            if result['sentiment'][i]==1:
                sy_count['normal'][j]=sy_count['normal'][j]+1
            if result['sentiment'][i]==0:
                sy_count['bad'][j]=sy_count['bad'][j]+1
            break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [29]:
#同义词替代
for i in range(len(sy_count)):
    for k in range(len(synos)):
        if synonyms.compare(sy_count['prop'][i], synos[k], seg=True)>0.8:
            sy_count['prop'][i]=synos[k]
            break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [30]:
#去重 排序
sy_count=sy_count.groupby('prop').sum()
sy_count=sy_count.sort_values(by="sum" , ascending=False)

In [32]:
sy_count

,sum,good,normal,bad
prop,,,,
感觉,8,8,0,0
外观,3,3,0,0
手机,3,3,0,0
速度快,2,2,0,0
商品,2,2,0,0
功能,2,2,0,0
确实,2,2,0,0
物流速度,1,1,0,0
超市,1,1,0,0
